In [ ]:
import sys, os, math
import sys
sys.path.insert(0, '..')
sys.path.insert(0, '../../quizx/pybindings')
import pyzx as zx
import quizx as qzx
import numpy as np
from pyzx import print_matrix
from pyzx.basicrules import *
import time
from fractions import Fraction

Z = zx.VertexType.Z
X = zx.VertexType.X
B = zx.VertexType.BOUNDARY
SE = zx.EdgeType.SIMPLE
HE = zx.EdgeType.HADAMARD

In [ ]:
c = zx.qasm("""
qreg q[4];
cx q[0], q[1];
cx q[2], q[1];
cx q[2], q[3];
cx q[3], q[0];
cx q[3], q[1];
cx q[2], q[1];
cx q[1], q[2];
""")
g = c.to_graph()
zx.draw(g, labels=True)

In [ ]:
fuse(g, 7, 9)

In [ ]:
zx.draw(g)

In [ ]:
g = c.to_graph()
while any(fuse(g, g.edge_s(e), g.edge_t(e)) for e in g.edges()): pass
zx.draw(g)

In [ ]:
def finished(g, v):
    return ((g.type(v) == X) and any(w in g.outputs() for w in g.neighbors(v)) or
            (g.type(v) == Z) and any(w in g.inputs()  for w in g.neighbors(v)))

def strong_comp_safe(g, v0, v1):
    if finished(g, v0) or finished(g, v1): return False
    else: return strong_comp(g, v0, v1)

def phase_free_simp(g):
    while (
        any(fuse(g, g.edge_s(e), g.edge_t(e)) for e in g.edges()) or
        any(strong_comp_safe(g, g.edge_s(e), g.edge_t(e)) for e in g.edges())
    ): pass

In [ ]:
g = c.to_graph()
phase_free_simp(g)
zx.draw(g)

In [ ]:
c = zx.generate.CNOT_HAD_PHASE_circuit(qubits=5, depth=400, clifford=True)
g = c.to_graph()

In [ ]:
zx.draw(g)

In [ ]:
zx.simplify.clifford_simp(g)
g.normalize()
zx.draw(g)

In [ ]:
c1 = zx.extract_circuit(g.copy(), up_to_perm=True)
zx.draw(c1)

In [ ]:
g = c.to_graph(backend="quizx-vec")

In [ ]:
zx.draw(g)

In [ ]:
gs = g.copy()
%time zx.simplify.clifford_simp(gs)
gs.normalize()
zx.draw(gs)

In [ ]:
gs = g.copy()
%time qzx.simplify.clifford_simp(gs)
gs.normalize()
zx.draw(gs)

In [ ]:
n = 50
g = 30
pfirst = 1/n
psecond = (1-pfirst)*1/(n-1)
pthird = (1-pfirst)*(1-psecond)*(1/(n-2))
p0 = pfirst + psecond + pthird
p = 1
for i in range(g): p *= 1-p0
p = 1-p
p